## 보팅방식 모델 구현 <hr>

-   데이터 : sklearn.database의 breast_cancer
-   유형 : 지도학습 + 분류
-   방법 : Voting 방식으로 진행 => LogisticRegression, DecisionTreeClassifier, SVC
-   학습데이터셋 : 동일한 데이터셋으로 3개의 모델로 학습 진행


[1] 모듈 로딩 및 데이터 준비


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer


In [39]:
X, y = load_breast_cancer(as_frame=True, return_X_y=True)


In [40]:
X.shape, y.shape


((569, 30), (569,))

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5, stratify=y
)


[2] 학습 진행 <hr>


[2-1] 앙상블 보팅 학습에 사용할 모델 인스턴스 생성


In [42]:
# LogisticRegression 인스턴스
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver="liblinear")
lr_model.fit(X_train, y_train)


LogisticRegression(solver='liblinear')

In [43]:
# DecisionTreeClassifier 인스턴스
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)


DecisionTreeClassifier()

In [44]:
# SVC 인스턴스
from sklearn.svm import SVC

svc_model = SVC(probability=True)
svc_model.fit(X_train, y_train)


SVC(probability=True)

[2-2] Ensemble 알고리즘 기반 분류


In [45]:
# 동일 데이터셋으로 병렬학습 진행할 모델 리스트 선정 및 결과 결정 방법 설정
from sklearn.ensemble import VotingClassifier

vt_models = VotingClassifier(
    estimators=[
        ("lr_model", lr_model),
        ("dt_model", dt_model),
        ("svc_model", svc_model),
    ],
    verbose=True,
)


In [46]:
# 동일 데이터셋을 전달해서 3개의 모델 동시에 학습 진행
vt_models.fit(X_train, y_train)


[Voting] ................. (1 of 3) Processing lr_model, total=   0.0s
[Voting] ................. (2 of 3) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 3) Processing svc_model, total=   0.0s


VotingClassifier(estimators=[('lr_model',
                              LogisticRegression(solver='liblinear')),
                             ('dt_model', DecisionTreeClassifier()),
                             ('svc_model', SVC(probability=True))],
                 verbose=True)

In [47]:
# 예측하기
new_data = pd.DataFrame([X_test.iloc[0]], columns=X_test.columns)
vt_models.predict(new_data)

# X_test.iloc[0].to_frame().T


array([1])

In [48]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법 (1)
vt_models.estimators_


[LogisticRegression(solver='liblinear'),
 DecisionTreeClassifier(),
 SVC(probability=True)]

In [49]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법 (2)
vt_models.named_estimators_.get("lr_model").predict(
    new_data
), vt_models.named_estimators_.get("dt_model").predict(
    new_data
), vt_models.named_estimators_.get(
    "svc_model"
).predict(
    new_data
)


(array([1], dtype=int64), array([1], dtype=int64), array([1], dtype=int64))

In [50]:
for key, value in vt_models.named_estimators_.items():
    print(f"{key} : {value.predict(new_data)[0]}")


lr_model : 1
dt_model : 1
svc_model : 1


-   3개 알고리즘 모델에 대한 성능 평가


In [52]:
print(
    f"[LogisticRegression] Train : {lr_model.score(X_train, y_train)}, Test : {lr_model.score(X_test, y_test)}"
)
print(
    f"[DecisionTreeClassifier] Train : {dt_model.score(X_train, y_train)}, Test : {dt_model.score(X_test, y_test)}"
)
print(
    f"[SVC] Train : {svc_model.score(X_train, y_train)}, Test : {svc_model.score(X_test, y_test)}"
)
print(
    f"[VotingClassifier] Train : {vt_models.score(X_train, y_train)}, Test : {vt_models.score(X_test, y_test)}"
)


[LogisticRegression] Train : 0.9582417582417583, Test : 0.9649122807017544
[DecisionTreeClassifier] Train : 1.0, Test : 0.9210526315789473
[SVC] Train : 0.9164835164835164, Test : 0.8771929824561403
[VotingClassifier] Train : 0.967032967032967, Test : 0.9298245614035088
